In [5]:
%pip install numpy
%pip install pandas
%pip install scikit-learn
%pip install kagglehub
%pip install --upgrade kagglehub
%pip install xgboost

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn.metrics import silhouette_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.ensemble import IsolationForest
import kagglehub
# same as ex1
# general setting. do not change TEST_SIZE
RANDOM_SEED = 42
TEST_SIZE = 0.3

# load dataset（from kagglehub）
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")
data = pd.read_csv(f"{path}/creditcard.csv")
data['Class'] = data['Class'].astype(int)

# prepare data
data = data.drop(['Time'], axis=1)
data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))

fraud = data[data['Class'] == 1]
nonfraud = data[data['Class'] == 0]
print(f'Fraudulent:{len(fraud)}, non-fraudulent:{len(nonfraud)}')
print(f'the positive class (frauds) percentage: {len(fraud)}/{len(fraud) + len(nonfraud)} ({len(fraud)/(len(fraud) + len(nonfraud))*100:.3f}%)')    


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Fraudulent:492, non-fraudulent:284315
the positive class (frauds) percentage: 492/284807 (0.173%)


In [ ]:

X = np.asarray(data.iloc[:, ~data.columns.isin(['Class'])])
Y = np.asarray(data.iloc[:, data.columns == 'Class'])

# split training set and data set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=TEST_SIZE, random_state=RANDOM_SEED)

#scaler = StandardScaler()
#x_train = scaler.fit_transform(x_train)
#x_test = scaler.transform(x_test)

#isoation forest
#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html
iso_forest = IsolationForest(contamination=0.0017, random_state=RANDOM_SEED)
iso_forest.fit(X_train)

#xgb model
#https://xgboost.readthedocs.io/en/latest/python/python_api.html ik i just linked the official doc, but it's the first one i saw and if you scroll down there is paramaters

xgb_model = XGBClassifier(
    colsample_bytree=1.0,
    learning_rate=0.1,
    max_depth=6,
    n_estimators=100,
    subsample=0.8,
    scale_pos_weight=2.5,
    eval_metric='logloss',
    tree_method='hist',
    random_state=RANDOM_SEED
)
xgb_model.fit(
    X_train, y_train,
    verbose = False,
    eval_set=[(X_train, y_train), (X_test, y_test)],
)

# same as ex1
def evaluation(y_true, y_pred, model_name="Model"):
   accuracy = accuracy_score(y_true, y_pred)
   precision = precision_score(y_true, y_pred, zero_division=0)
   recall = recall_score(y_true, y_pred)
   f1 = f1_score(y_true, y_pred)

   print(f'\n{model_name} Evaluation:')
   print('===' * 15)
   print('         Accuracy:', accuracy)
   print('  Precision Score:', precision)
   print('     Recall Score:', recall)
   print('         F1 Score:', f1)
   print("\nClassification Report:")
   print(classification_report(y_true, y_pred))

y_pred = xgb_model.predict(X_test)
evaluation(y_test, y_pred, model_name="isolation+XGBoost")
#did i use AI(llm)? yes. Why? i installed kagglehub 4 times on my machine ICANT  also they are not helpful, just looking at the example code is better than using llm(s) to get the answer, unless with sufficent prompt engi.

f:\ntcuml\.conda\Lib\site-packages\xgboost\training.py:183: UserWarning: [23:25:45] WARNING: D:\bld\xgboost-split_1748169168709\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



isolation+XGBoost Evaluation:
         Accuracy: 0.9996957035684608
  Precision Score: 0.9508196721311475
     Recall Score: 0.8529411764705882
         F1 Score: 0.8992248062015504

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85307
           1       0.95      0.85      0.90       136

    accuracy                           1.00     85443
   macro avg       0.98      0.93      0.95     85443
weighted avg       1.00      1.00      1.00     85443

